In [1]:
import os
from pathlib import Path

from arcgis import GIS, GeoAccessor
from dotenv import find_dotenv, load_dotenv
from modeling import Country, ModelingAccessor

load_dotenv(find_dotenv())

# load the "autoreload" extension so as src code is changed, the changes are picked up in the dataframe
%load_ext autoreload
%autoreload 2

In [2]:
project_parent = Path('.').absolute().parent.parent
dir_data = project_parent/'data'
dir_int = dir_data/'interim'
dir_raw = dir_data/'raw'
gdb_int = dir_int/'interim.gdb'
gdb_raw = dir_raw/'raw.gdb'

biz_drop_cols = ['OBJECTID', 'CONAME','SALESVOL', 'HDBRCH', 'ULTNUM', 'PUBPRV', 'EMPNUM', 'FRNCOD', 'ISCODE', 'SQFTCODE', 'LOC_NAME', 'STATUS', 'SCORE', 'SOURCE', 'REC_TYPE']

In [3]:
gis = GIS(os.getenv('ESRI_PORTAL_URL'), username=os.getenv('ESRI_PORTAL_USERNAME'), password=os.getenv('ESRI_PORTAL_PASSWORD'))
usa = Country('USA')

usa

<modeling.Country - USA (GIS at https://geoai-ent.bd.esri.com/portal/ logged in as jmccune)>

In [105]:
df = usa.enrich_variables.loc[:,['name', 'alias', 'description', 'data_collection']]

df

,name,alias,description,data_collection
0,AGE0_CY,2020 Population Age <1,2020 Total Population Age <1 (Esri),1yearincrements
1,AGE1_CY,2020 Population Age 1,2020 Total Population Age 1 (Esri),1yearincrements
2,AGE2_CY,2020 Population Age 2,2020 Total Population Age 2 (Esri),1yearincrements
3,AGE3_CY,2020 Population Age 3,2020 Total Population Age 3 (Esri),1yearincrements
4,AGE4_CY,2020 Population Age 4,2020 Total Population Age 4 (Esri),1yearincrements
...,...,...,...,...
37,MOEMEDYRMV,2018 Median Year Householder Moved In MOE (ACS...,2018 Median Year Householder Moved into Unit M...,yearmovedin
38,RELMEDYRMV,2018 Median Year Householder Moved In REL (ACS...,2018 Median Year Householder Moved into Unit R...,yearmovedin
39,ACSOWNER,2018 Owner Households (ACS 5-Yr),2018 Owner Households (ACS 5-Yr),yearmovedin
40,MOEOWNER,2018 Owner Households MOE (ACS 5-Yr),2018 Owner Households MOE (ACS 5-Yr),yearmovedin


In [109]:
groupby_column = 'name'
aggregate_column = 'data_collection'

agg_df = df.groupby(groupby_column).aggregate({aggregate_column: list})

agg_df

,data_collection
name,
ACS65HI2MM,"[Health, healthinsurancecoverage]"
MALE35_FY,"[5yearincrements, gender]"
MOEMEDCRNT,"[ACS_Housing_Summary_rep, housingcosts]"
N38_BUS,"[businesses, industrybynaicscode]"
REL35DIRHI,"[ACS_Population_Summary_rep, Health, healthins..."
REL35VAHCR,"[ACS_Population_Summary_rep, Health, healthins..."
X2001_I,"[Household_Budget_Expenditures_rep, Retail_Goo..."
X4037_I,"[House_and_Home_Expenditures_rep, HousingHouse..."
X4061_I,"[House_and_Home_Expenditures_rep, HousingHouse..."


In [107]:
ex_vars = agg_vars[agg_vars[aggregate_column].apply(lambda val: len(val) > 1)]
                   
ex_vars

,data_collection
name,
ACS65HI2MM,"[Health, healthinsurancecoverage]"
MALE35_FY,"[5yearincrements, gender]"
MOEMEDCRNT,"[ACS_Housing_Summary_rep, housingcosts]"
N38_BUS,"[businesses, industrybynaicscode]"
REL35DIRHI,"[ACS_Population_Summary_rep, Health, healthins..."
REL35VAHCR,"[ACS_Population_Summary_rep, Health, healthins..."
X2001_I,"[Household_Budget_Expenditures_rep, Retail_Goo..."
X4037_I,"[House_and_Home_Expenditures_rep, HousingHouse..."
X4061_I,"[House_and_Home_Expenditures_rep, HousingHouse..."


In [108]:
df = df[df.name.isin(ex_vars.index)].reset_index(drop=True)

df

,name,alias,description,data_collection
0,MALE35_FY,2025 Males Age 35-39,2025 Male Population Age 35-39 (Esri),5yearincrements
1,MOEMEDCRNT,2018 Median Contract Rent MOE (ACS 5-Yr),2018 Median Contract Rent (Households Paying C...,ACS_Housing_Summary_rep
2,REL35DIRHI,2018 Pop 35-64: Direct-Purch Health Ins REL (A...,2018 Population 35-64: Direct-Purchase Health ...,ACS_Population_Summary_rep
3,REL35VAHCR,2018 Pop 35-64: VA Health Care Only REL (ACS 5...,2018 Population 35-64: VA Health Care Only REL...,ACS_Population_Summary_rep
4,REL35DIRHI,2018 Pop 35-64: Direct-Purch Health Ins REL (A...,2018 Population 35-64: Direct-Purchase Health ...,Health
5,REL35VAHCR,2018 Pop 35-64: VA Health Care Only REL (ACS 5...,2018 Population 35-64: VA Health Care Only REL...,Health
6,ACS65HI2MM,2018 Pop 65+: Medicare & Medicaid (ACS 5-Yr),2018 Population 65+: Medicare & Medicaid Cover...,Health
7,X4037_I,2020 Index: Housekeeping Supply-Other HH Product,2020 Housekeeping Supplies - Other HH Products...,House_and_Home_Expenditures_rep
8,X4061_I,2020 Index: Wall Units/Cabinets/Other LR/FR/RR...,2020 Furniture -Wall Units/Cabinets/Other LR/F...,House_and_Home_Expenditures_rep
9,X4072_I,2020 Index: Sewing Machines/Misc Appliances,2020 Major Appliances - Sewing Machines/Misc A...,House_and_Home_Expenditures_rep


In [110]:
ex_df = evars[evars.name.isin(ex_vars.index)].reset_index(drop=True)

ex_df

,name,alias,description,data_collection
0,MALE35_FY,2025 Males Age 35-39,2025 Male Population Age 35-39 (Esri),5yearincrements
1,MOEMEDCRNT,2018 Median Contract Rent MOE (ACS 5-Yr),2018 Median Contract Rent (Households Paying C...,ACS_Housing_Summary_rep
2,REL35DIRHI,2018 Pop 35-64: Direct-Purch Health Ins REL (A...,2018 Population 35-64: Direct-Purchase Health ...,ACS_Population_Summary_rep
3,REL35VAHCR,2018 Pop 35-64: VA Health Care Only REL (ACS 5...,2018 Population 35-64: VA Health Care Only REL...,ACS_Population_Summary_rep
4,REL35DIRHI,2018 Pop 35-64: Direct-Purch Health Ins REL (A...,2018 Population 35-64: Direct-Purchase Health ...,Health
5,REL35VAHCR,2018 Pop 35-64: VA Health Care Only REL (ACS 5...,2018 Population 35-64: VA Health Care Only REL...,Health
6,ACS65HI2MM,2018 Pop 65+: Medicare & Medicaid (ACS 5-Yr),2018 Population 65+: Medicare & Medicaid Cover...,Health
7,X4037_I,2020 Index: Housekeeping Supply-Other HH Product,2020 Housekeeping Supplies - Other HH Products...,House_and_Home_Expenditures_rep
8,X4061_I,2020 Index: Wall Units/Cabinets/Other LR/FR/RR...,2020 Furniture -Wall Units/Cabinets/Other LR/F...,House_and_Home_Expenditures_rep
9,X4072_I,2020 Index: Sewing Machines/Misc Appliances,2020 Major Appliances - Sewing Machines/Misc A...,House_and_Home_Expenditures_rep


In [111]:
df_alias = df.drop(columns=aggregate_column).set_index(groupby_column)

out_df = agg_df.join(df_alias).reset_index(groupby_column).drop_duplicates(groupby_column).reset_index(drop=True)

print(out_df.to_markdown())

|    | name       | data_collection                                                                             | alias                                                    | description                                                            |
|---:|:-----------|:--------------------------------------------------------------------------------------------|:---------------------------------------------------------|:-----------------------------------------------------------------------|
|  0 | ACS65HI2MM | ['Health', 'healthinsurancecoverage']                                                       | 2018 Pop 65+: Medicare & Medicaid (ACS 5-Yr)             | 2018 Population 65+: Medicare & Medicaid Coverage (ACS 5-Yr)           |
|  1 | MALE35_FY  | ['5yearincrements', 'gender']                                                               | 2025 Males Age 35-39                                     | 2025 Male Population Age 35-39 (Esri)                                  |
|  2 | MOEMEDCRN

In [101]:
import pandas as pd

def aggregate_column(df: pd.DataFrame, groupby_column: str = 'name', 
                     aggregate_column: str = 'data_collection') -> pd.DataFrame:
    
    # make sure the columns are in the dataframe
    assert groupby_column in df.columns, f'"groupby_column", {groupby_column}, '
                                         'does not appear to be in the input '
                                         'DataFrame columns.'
    assert aggregate_column in df.columns, f'"aggregate_column", {aggregate_column}, '
                                           'does not appear to be in the input '
                                           'DataFrame columns.'
    
    # create aggregated dataframe
    agg_df = df.groupby(groupby_column).aggregate({aggregate_column: list})
    
    # create a dataframe of values without the aggregation column ready to join
    df_alias = 	df.drop(columns=aggregate_column).\
        set_index(groupby_column)
    
    # join the aliases on and clean up the result
    out_df = agg_df.join(df_alias).\
        reset_index(groupby_column).\
        drop_duplicates(groupby_column).\
        reset_index(drop=True)
    
    return out_df

In [39]:
evars = ex_df

agg_vars = evars.groupby(groupby_column).aggregate({aggregate_column: list})

agg_vars

,data_collection
name,
ACS65HI2MM,"[Health, healthinsurancecoverage]"
MALE35_FY,"[5yearincrements, gender]"
MOEMEDCRNT,"[ACS_Housing_Summary_rep, housingcosts]"
N38_BUS,"[businesses, industrybynaicscode]"
REL35DIRHI,"[ACS_Population_Summary_rep, Health, healthins..."
REL35VAHCR,"[ACS_Population_Summary_rep, Health, healthins..."
X2001_I,"[Household_Budget_Expenditures_rep, Retail_Goo..."
X4037_I,"[House_and_Home_Expenditures_rep, HousingHouse..."
X4061_I,"[House_and_Home_Expenditures_rep, HousingHouse..."


In [5]:
source = usa.source
prop = source._con.get(source.properties.helperServices.route.url)
prop

{'currentVersion': 10.81,
 'layerName': 'Route',
 'layerType': 'esriNAServerRouteLayer',
 'impedance': 'TravelTime',
 'restrictions': ['Avoid Unpaved Roads',
  'Avoid Private Roads',
  'Driving an Automobile',
  'Through Traffic Prohibited',
  'Roads Under Construction Prohibited',
  'Avoid Gates',
  'Avoid Express Lanes',
  'Avoid Carpool Roads'],
 'snapTolerance': 0,
 'maxSnapTolerance': 20000,
 'snapToleranceUnits': 'esriMeters',
 'ignoreInvalidLocations': True,
 'restrictUTurns': 'esriNFSBAtDeadEndsAndIntersections',
 'accumulateAttributeNames': ['Miles', 'Kilometers'],
 'attributeParameterValues': [{'attributeName': 'Avoid Limited Access Roads',
   'parameterName': 'Restriction Usage',
   'parameterType': 'float',
   'value': 'Avoid_Medium'},
  {'attributeName': 'Avoid Ferries',
   'parameterName': 'Restriction Usage',
   'parameterType': 'float',
   'value': 'Avoid_Medium'},
  {'attributeName': 'Avoid Unpaved Roads',
   'parameterName': 'Restriction Usage',
   'parameterType': 'f

In [16]:
%%time
aoi_df = usa.cbsas.get('seattle')

aoi_df

CPU times: user 43.1 ms, sys: 7.14 ms, total: 50.2 ms
Wall time: 500 ms


,ID,NAME,SHAPE
0,42660,"Seattle-Tacoma-Bellevue, WA Metropolitan Stati...","{""rings"": [[[-121.68612849977673, 48.298988999..."


In [5]:
%%time
bg_df = aoi_df.dm.level(0).get()

bg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2478 entries, 0 to 2477
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   ID      2478 non-null   object  
 1   NAME    2478 non-null   object  
 2   SHAPE   2478 non-null   geometry
dtypes: geometry(1), object(2)
memory usage: 58.2+ KB
Wall time: 14 s


In [6]:
%%time
biz_df = usa.business.get_by_name('ace hardware', aoi_df).drop(columns=biz_drop_cols)
biz_df.spatial.set_geometry('SHAPE')

biz_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   LOCNUM               33 non-null     object  
 1   STREET               33 non-null     object  
 2   CITY                 33 non-null     object  
 3   STATE                33 non-null     object  
 4   STATE_NAME           33 non-null     object  
 5   ZIP                  33 non-null     object  
 6   ZIP4                 33 non-null     object  
 7   NAICS                33 non-null     object  
 8   SIC                  33 non-null     object  
 9   SHAPE                33 non-null     geometry
 10  id                   33 non-null     object  
 11  brand_name           33 non-null     object  
 12  brand_name_category  33 non-null     object  
dtypes: geometry(1), object(12)
memory usage: 3.5+ KB
Wall time: 2.54 s


In [7]:
%%time
bg_near_biz_df = bg_df.dm.get_nearest(biz_df, origin_id_column='ID', near_prefix='brand')

bg_near_biz_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2478 entries, 0 to 2477
Data columns (total 23 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   ID                                    2478 non-null   object  
 1   NAME                                  2478 non-null   object  
 2   SHAPE                                 2478 non-null   geometry
 3   brand_destination_id_01               2476 non-null   object  
 4   brand_proximity_kilometers_01         2476 non-null   float64 
 5   brand_proximity_minutes_01            2476 non-null   float64 
 6   brand_proximity_side_street_left_01   2476 non-null   float64 
 7   brand_proximity_side_street_right_01  2476 non-null   float64 
 8   brand_destination_id_02               2476 non-null   object  
 9   brand_proximity_kilometers_02         2476 non-null   float64 
 10  brand_proximity_minutes_02            2476 non-null   float64 
 11  bran

In [8]:
%%time
comp_df = usa.business.get_competition(biz_df, aoi_df, local_threshold=3).drop(columns=biz_drop_cols)
comp_df.spatial.set_geometry('SHAPE')

comp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   LOCNUM               76 non-null     object  
 1   STREET               76 non-null     object  
 2   CITY                 76 non-null     object  
 3   STATE                76 non-null     object  
 4   STATE_NAME           76 non-null     object  
 5   ZIP                  76 non-null     object  
 6   ZIP4                 76 non-null     object  
 7   NAICS                76 non-null     object  
 8   SIC                  76 non-null     object  
 9   SHAPE                76 non-null     geometry
 10  id                   76 non-null     object  
 11  brand_name           76 non-null     object  
 12  brand_name_category  76 non-null     object  
dtypes: geometry(1), object(12)
memory usage: 7.8+ KB
Wall time: 2.37 s


In [9]:
%%time
bg_near_biz_comp_df = bg_near_biz_df.dm.get_nearest(comp_df, origin_id_column='ID', near_prefix='comp', 
                                                    destination_columns_to_keep=['brand_name', 'brand_name_category'])

bg_near_biz_comp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2478 entries, 0 to 2477
Data columns (total 51 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   ID                                    2478 non-null   object  
 1   NAME                                  2478 non-null   object  
 2   SHAPE                                 2478 non-null   geometry
 3   brand_destination_id_01               2476 non-null   object  
 4   brand_proximity_kilometers_01         2476 non-null   float64 
 5   brand_proximity_minutes_01            2476 non-null   float64 
 6   brand_proximity_side_street_left_01   2476 non-null   float64 
 7   brand_proximity_side_street_right_01  2476 non-null   float64 
 8   brand_destination_id_02               2476 non-null   object  
 9   brand_proximity_kilometers_02         2476 non-null   float64 
 10  brand_proximity_minutes_02            2476 non-null   float64 
 11  bran

In [10]:
bg_near_biz_comp_df.iloc[0]

ID                                                                           530530701003
NAME                                                                        530530701.003
SHAPE                                   {'rings': [[[-122.1652430000312, 47.0830489997...
brand_destination_id_01                                                         677129595
brand_proximity_kilometers_01                                                     40.8306
brand_proximity_minutes_01                                                        41.3118
brand_proximity_side_street_left_01                                                     0
brand_proximity_side_street_right_01                                                    1
brand_destination_id_02                                                         371889957
brand_proximity_kilometers_02                                                     42.8784
brand_proximity_minutes_02                                                        43.5606
brand_prox

In [21]:
bg_near_biz_comp_df.to_csv(dir_raw/'prox_cust.csv')